In [1]:
import pandas as pd
import altair as alt
import numpy as np

In [2]:
def flipColNames(c):
    d = dict()
    for s in c:
        parts = s.split("_")
        d[s] = "_".join(parts[::-1])
    return d

In [3]:
#load predictions and lines
df_p = pd.read_pickle(f"./logs/current_and_future_predictions.pkl")
df_p = df_p.loc[df_p['FUTURE_GAME']==1] #get only upcoming predictions

lines = pd.read_pickle(f"./logs/latest_line_all.pkl")

In [4]:
#get labels for the stats and stats+"_PREDICT"
stats = ["PTS", "REB", "AST", "FG3M", "BLK", "STL"]
stats_pred = [s+"_PREDICT" for s in stats]

In [5]:
#join the lines and the predictions
non_stat_cols = ["PLAYER_NAME",
         "OPPONENT",
         "GAME_DATE"]

df_p = (df_p[non_stat_cols+['POSITION']+stats_pred]
        .groupby(non_stat_cols).agg("mean").reset_index()
        .merge(lines, left_on='PLAYER_NAME', right_on='PLAYER')
        .drop("PLAYER", axis=1)
       )

In [6]:
#make a normalized deltaF/F "score" for comparing deviations among stats 
#e.g. +3 BLK predicted is much stronger signal than +3 PTS

for stat in stats:
    df_p[stat+"_dff"] = (df_p[stat+"_PREDICT"] - df_p[stat+"_LINE"]) / df_p[stat+"_LINE"]

In [14]:
#flip the column names for proper conversion to long
c = df_p.columns.to_list()[3:]
c_flip = flipColNames(c)
df_p.rename(columns=c_flip, inplace=True)

In [19]:
labels = ["PREDICT", "LINE", "OVER", "UNDER", "dff"]
df_p_long = pd.wide_to_long(df_p, stubnames=labels, i = ["PLAYER_NAME","GAME_DATE"], j="STAT", sep="_", suffix=r'\w+')[labels]
df_p_long.reset_index(inplace=True)

In [20]:
df_p_long.to_pickle(f"./logs/")